In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pandas as pd

ModuleNotFoundError: No module named 'selenium'

In [4]:
def getTitles(end_page): 
    result = {}
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Connection': 'keep-alive',
    }
    for i in range(1, end_page):
        url = "https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&newsearch=true&highlight=true&returnFacets=ALL&returnType=SEARCH&matchPubs=true&rowsPerPage=100&pageNumber=" + str(i)
        response = requests.get(url, headers=headers)
        response = response.content
        soup = BeautifulSoup(response, 'html.parser')
        print(soup)
        all_books_on_page = soup.find('div', id='xplMainContent')
        print(all_books_on_page)
        books = all_books_on_page.find_all('h3')
        for book in books:
            title = book.find('a')
            name = title.text
            link = title.attrs['href']
            result[name] = "https://ieeexplore.ieee.org" + link
    return result

In [5]:
getTitles(3)

<!DOCTYPE html>

<apm_do_not_touch>
<script type="text/javascript">
(function(){
window.WVh=!!window.WVh;try{(function(){(function(){var a={decrypt:function(d){try{return JSON.parse(function(g){g=g.split("l");var h="";for(var k=0;k<g.length;++k)h+=String.fromCharCode(g[k]);return h}(d))}catch(g){}}};return a={configuration:a.decrypt("123l34l97l99l116l105l118l101l34l58l34l110l111l34l44l34l100l101l98l117l103l103l105l110l103l34l58l34l110l111l34l44l34l109l111l100l117l108l101l49l34l58l34l101l110l97l98l108l101l100l34l44l34l109l111l100l117l108l101l50l34l58l34l101l110l97l98l108l101l100l34l44l34l109l111l100l117l108l101l51l34l58l34l101l110l97l98l108l101l100l34l44l34l109l111l100l117l108l101l52l34l58l34l101l110l97l98l108l101l100l34l125")}})();
var b=86;try{var da,ma,qa=c(751)?0:1,ra=c(116)?1:0,sa=c(916)?0:1,ua=c(995)?0:1;for(var ya=(c(782),0);ya<ma;++ya)qa+=c(960)?1:2,ra+=c(993)?1:2,sa+=(c(473),2),ua+=c(217)?3:1;da=qa+ra+sa+ua;window.Qa===da&&(window.Qa=++da)}catch(a){window.Qa=da}var e=!0;functio

AttributeError: 'NoneType' object has no attribute 'find_all'

In [6]:

def getTitles(end_page): 
    result = {}
    options = webdriver.ChromeOptions()
    # Set up Selenium WebDriver (e.g., using Chrome)
    driver = webdriver.Chrome(options=options)
    
    for i in range(1, end_page):
        url = f"https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&newsearch=true&highlight=true&returnFacets=ALL&returnType=SEARCH&matchPubs=true&rowsPerPage=100&pageNumber={i}"
        driver.get(url)
        
        # Wait for the page to load
        time.sleep(5)  # Adjust the sleep time based on your network speed
        
        # Get page source after JavaScript has executed
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        all_books_on_page = soup.find('xpl-results-list')
        if all_books_on_page:
            books = all_books_on_page.find_all('h3')
            for book in books:
                title = book.find('a')
                if title:
                    name = title.text.strip()
                    link = title.attrs['href']
                    result[name] = "https://ieeexplore.ieee.org" + link
        else:
            print(f"No 'xpl-results-list' found on page {i}.")
    
    driver.quit()
    return result

titles = getTitles(3)
print(titles)


NameError: name 'time' is not defined

In [ ]:
def getDescriptions(url_list):
    options = webdriver.ChromeOptions()
    # Launch Chrome:
    driver = webdriver.Chrome(options=options)
    result = []
    for i in url_list:
        driver.get(str(url_list[i]))
        try:
            WebDriverWait(driver, 10).until_not(
                EC.text_to_be_present_in_element(
                    (By.ID, 'descriptionPlaceholder'),
                    'Loading Description...'
                )
            )
        except TimeoutException:
            print(f"Timeout while waiting for description to load on {str(url_list[i])}")
            continue
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        dynamic_content = soup.find('div', id='descriptionPlaceholder')
        result.append(dynamic_content.get_text(strip=True))
    driver.quit()
    return result

In [7]:
def getTitles(end_page):
    options = webdriver.ChromeOptions()
    # Launch Chrome:
    driver = webdriver.Chrome(options=options)
    result = {}
    for i in range(1, end_page):
        url = "https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&newsearch=true&highlight=true&returnFacets=ALL&returnType=SEARCH&matchPubs=true&rowsPerPage=10&pageNumber=" + str(i)
        driver.get(url)
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CLASS_NAME, 'List-results-items')
                )
            )
        except TimeoutException:
            print(f"Timeout while waiting for page to load ")
            continue
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        content = soup.find('xpl-results-list')
        articles = content.find_all('h3')
        for article in articles:
            title = article.find('a')
            name = title.text
            link = title.attrs['href']
            result[name] = "https://ieeexplore.ieee.org" + link
    driver.quit()
    return result



In [8]:
titles = getTitles(2)

In [9]:
titles

{'Deep Residual Learning for Image Recognition': 'https://ieeexplore.ieee.org/document/7780459/',
 'A mathematical theory of communication': 'https://ieeexplore.ieee.org/document/6773024/',
 'A new look at the statistical model identification': 'https://ieeexplore.ieee.org/document/1100705/',
 'Image quality assessment: from error visibility to structural similarity': 'https://ieeexplore.ieee.org/document/1284395/',
 'Gradient-based learning applied to document recognition': 'https://ieeexplore.ieee.org/document/726791/',
 'ImageNet: A large-scale hierarchical image database': 'https://ieeexplore.ieee.org/document/5206848/',
 'A fast and elitist multiobjective genetic algorithm: NSGA-II': 'https://ieeexplore.ieee.org/document/996017/',
 'Particle swarm optimization': 'https://ieeexplore.ieee.org/document/488968/',
 'Going deeper with convolutions': 'https://ieeexplore.ieee.org/document/7298594/',
 'You Only Look Once: Unified, Real-Time Object Detection': 'https://ieeexplore.ieee.org/d

In [12]:
def getInfo(url_list):
    options = webdriver.ChromeOptions()
    # Launch Chrome:
    driver = webdriver.Chrome(options=options)
    result = []
    for i in url_list:
        driver.get(str(url_list[i]))
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CLASS_NAME, 'u-mb-1')
                )
            )
        except TimeoutException:
            print(f"Timeout while waiting for abstract to load on {str(url_list[i])}")
            continue
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        dynamic_content = soup.find('div', {'xplmathjax': True})
        result.append(dynamic_content.get_text(strip=True))
        # articles = content.find_all('h3')
        # for article in articles:
        #     title = article.find('a')
        #     name = title.text
        #     link = title.attrs['href']
        #     result[name] = "https://ieeexplore.ieee.org" + link
    driver.quit()
    return result

In [29]:
def getInfo(url_list):
    options = webdriver.ChromeOptions()
    # Launch Chrome:
    driver = webdriver.Chrome(options=options)
    result = {}
    for i in url_list:
        driver.get(str(url_list[i]))
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CLASS_NAME, 'u-mb-1')
                )
            )
        except TimeoutException:
            print(f"Timeout while waiting for abstract to load on {str(url_list[i])}")
            continue
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        title = soup.find('h1', class_="document-title text-2xl-md-lh")
        title = title.get_text(strip=True)
        # dynamic_content = soup.find('div', class_='abstract-desktop-div')
        description = soup.find('div', {'xplmathjax': True})
        description = description.get_text(strip=True)
        authors = soup.find_all('a', triggers='hover')
        author_names = [a.get_text(strip=True) for a in authors]
        



        result[title] = [description, author_names]
        
        # articles = content.find_all('h3')

        # title = article.find('a')
        # name = title.text
        # link = title.attrs['href']
        # result[name] = "https://ieeexplore.ieee.org" + link
    driver.quit()
    return result

In [30]:
getInfo(titles)

{'Deep Residual Learning for Image Recognition': ['Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers - 8× deeper than VGG nets [40] but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers. The depth of representations is of central importance for many visual recognition tasks. So